<a href="https://colab.research.google.com/github/DaeSeokSong/image-processing/blob/feature%2FUnet-scar/Processor_PerformanceTester_Scar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import**

In [ ]:
# Image processing
import cv2
import numpy as np

from google.colab.patches import cv2_imshow
from google.colab import output

# ETC
import os
import time
import math

## Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Models/GAN_Scar
!ls -al

/content/gdrive/MyDrive/Models/GAN_Scar
total 175
drwx------ 2 root root  4096 Aug 16 08:53  Dataset
-rw------- 1 root root 22891 Sep  1 12:04  Image_segmentation-Scar.ipynb
drwx------ 2 root root  4096 Aug 17 05:49  Log
-rw------- 1 root root 49019 Sep  2 04:06  Processor_PerformanceTester-Scar.ipynb
drwx------ 2 root root  4096 Aug 16 08:39  Raw_Dataset
-rw------- 1 root root 16167 Sep  1 12:13  RawDataset_Processor-Scar.ipynb
drwx------ 2 root root  4096 Aug 23 14:47  result
-rw------- 1 root root 39995 Aug 15 11:40 'UNet architecture.PNG'
-rw------- 1 root root 33422 Sep  2 15:31  Unet-Scar.ipynb


# **Grobal variable**

In [ ]:
# Path
MODEL_PATH = "/content/gdrive/MyDrive/Models/GAN_Scar"

RAW_TRAIN_SET_PATH = "/Raw_Dataset/train"
RAW_TEST_SET_PATH = "/Raw_Dataset/test"

DATASET_PATH = "/Dataset"

IMAGES_PATH = '/images'
LABELS_PATH = '/labels'

# Image preprocess
NORM_INPUT_W_SIZE = 572
NORM_INPUT_H_SIZE = 572

# **Function**

In [ ]:
def imshow_waitkey_enter(image):
    cv2_imshow(image)

    time.sleep(0.5)
    
    input("Please press the Enter key to proceed\n")
    output.clear()

    pass

# **Make dir**

In [ ]:
# Create processed dataset dir
train_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'train')
val_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'val')
test_dir = os.path.join(MODEL_PATH + DATASET_PATH, 'test')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)

if not os.path.exists(val_dir):
    os.makedirs(val_dir)

if not os.path.exists(test_dir):
    os.makedirs(test_dir)

# 사용 모델

## UNet

**마지막 Output 채널 2 → 1 변경**

<img src = "https://drive.google.com/uc?id=14CzAAaKv5v7pVfvugBRbD1xI4IuhmoyT"  width = 640>


<br>
<br>
<br>

# ***데이터셋 학습 비교 실험***


---

<br>

***공통사항***
* 원본 이미지 크기 = 244 * 244
* epoch = 40
* batch = 8
> UNet batch size = 4 <br>
> Wound dataset batch size = 8
* lr = 1e-3

<br>

***주의사항***
* 이미지의 사이즈는 정규화 되어야 함

<br>

***참고사항***
* CNN 대표 모델들의 네트워크 입력 사이즈는 224 * 224
* 이미지의 사이즈가 큰 경우에, Overlap-Tite(down sampling) 전략으로 한 이미지를 나눠서 학습시킨다.

<br>

---




<br>
<br>
<br>

## [대조군](https://www.nature.com/articles/s41598-020-78799-w)

### Performance

### Process dataset

#### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

#### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

<br>
<br>
<br>

## 실험군

### 1

*적용사항*
1. Lab space를 이용한 Contrast 조정
2. BGR의 Green channel 사용

#### Performance

```
Input size =  torch.Size([8, 1, 224, 224])

TRAIN || EPOCH 0020 | BATCH  069 /  073 | LOSS  0.6762
TRAIN || EPOCH 0020 | BATCH  070 /  073 | LOSS  0.6762
TRAIN || EPOCH 0020 | BATCH  071 /  073 | LOSS  0.6762
TRAIN || EPOCH 0020 | BATCH  072 /  073 | LOSS  0.6762
TRAIN || EPOCH 0020 | BATCH  073 /  073 | LOSS  0.6762

VALID || EPOCH 0020 | BATCH  028 /  032 | LOSS  0.6805
VALID || EPOCH 0020 | BATCH  029 /  032 | LOSS  0.6805
VALID || EPOCH 0020 | BATCH  030 /  032 | LOSS  0.6806
VALID || EPOCH 0020 | BATCH  031 /  032 | LOSS  0.6807
VALID || EPOCH 0020 | BATCH  032 /  032 | LOSS  0.6805
```

<img src = "https://drive.google.com/uc?id=1GOlvg4VIjbExmrc3OK0qB90d1_fFt0Q7"  width = 400>

<img src = "https://drive.google.com/uc?id=1AvmMcgoEUfnPNYgMO4WW6iYiynyJKHE3"  width = 800>

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 2

*적용사항*
1. 불필요 Black boundary 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Resize input to 572*572
5. Resize output to 388*388
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

```
Input size =  torch.Size([8, 1, 572, 572])

TRAIN || EPOCH 0020 | BATCH  069 /  073 | LOSS  0.5164
TRAIN || EPOCH 0020 | BATCH  070 /  073 | LOSS  0.5159
TRAIN || EPOCH 0020 | BATCH  071 /  073 | LOSS  0.5155
TRAIN || EPOCH 0020 | BATCH  072 /  073 | LOSS  0.5150
TRAIN || EPOCH 0020 | BATCH  073 /  073 | LOSS  0.5146

VALID || EPOCH 0020 | BATCH  028 /  032 | LOSS  0.5083
VALID || EPOCH 0020 | BATCH  029 /  032 | LOSS  0.5070
VALID || EPOCH 0020 | BATCH  030 /  032 | LOSS  0.5049
VALID || EPOCH 0020 | BATCH  031 /  032 | LOSS  0.5052
VALID || EPOCH 0020 | BATCH  032 /  032 | LOSS  0.5043
```

<img src = "https://drive.google.com/uc?id=1uiIqGtvEmYWUhBbjKwjDtncDfg66b1cw"  width = 400>

<img src = "https://drive.google.com/uc?id=1Q5tzGUKwYX5rX4Viy12zHBPukUx1AjtO"  width = 800>

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Resize image
    green_image = resize_image(green_image)
    label_image = resize_image(label_image)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 3

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Bordering(cv2.copymakeborder CONSTANT) input to 572*572
5. Resize output to 388*388
> UNet 아키텍쳐 입력 이미지 사이즈

##### Performance

```
Input size =  torch.Size([8, 1, 572, 572])

TRAIN || EPOCH 0020 | BATCH  069 /  073 | LOSS  0.3800
TRAIN || EPOCH 0020 | BATCH  070 /  073 | LOSS  0.3800
TRAIN || EPOCH 0020 | BATCH  071 /  073 | LOSS  0.3800
TRAIN || EPOCH 0020 | BATCH  072 /  073 | LOSS  0.3800
TRAIN || EPOCH 0020 | BATCH  073 /  073 | LOSS  0.3800

VALID || EPOCH 0020 | BATCH  028 /  032 | LOSS  0.3811
VALID || EPOCH 0020 | BATCH  029 /  032 | LOSS  0.3811
VALID || EPOCH 0020 | BATCH  030 /  032 | LOSS  0.3811
VALID || EPOCH 0020 | BATCH  031 /  032 | LOSS  0.3810
VALID || EPOCH 0020 | BATCH  032 /  032 | LOSS  0.3810
```

<img src = "https://drive.google.com/uc?id=170ISySZSU8JKrOss--9rPujTHRf7TI0i"  width = 400>

<img src = "https://drive.google.com/uc?id=1g3Wbe6ym3VK0srroYD-bnwSbMy9QYLW5"  width = 800>

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{val_idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{val_idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    image = cv2.copyMakeBorder(image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_image = cv2.equalizeHist(l_image)
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    b_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(16, 16))
    a_image = a_clahe.apply(a_image)
    b_image = b_clahe.apply(b_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    blue_image, green_image, red_image = cv2.split(enhanced_image)

    green_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    green_image = green_clahe.apply(green_image)

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), green_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)

### 4

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. BGR의 Green channel 사용
4. Resize input, label image width 또는 heigth 512 (비율 그대로)
5. Bordering(cv2.copymakeborder CONSTANT) input to 572*572
6. Resize output to 388*388
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Resize images

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    if width > height: height = width
    else: width = height

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # resize

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # resize

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

### 5

*적용사항*
1. Black boundary 모두 제거
2. Lab space를 이용한 CLAHE Contrast 조정
3. K-Means(k=3)를 이용한 이미지 분류
4. Resize input, label image width 또는 heigth 512 (비율 그대로)
5. Bordering(cv2.copymakeborder CONSTANT) input to 572*572
6. Resize output to 388*388
> UNet 아키텍쳐 입력 이미지 사이즈

#### Performance

#### Process dataset

##### Train

In [ ]:
# Set load image dir path
image_path = MODEL_PATH + RAW_TRAIN_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TRAIN_SET_PATH + LABELS_PATH

# Load raw train images
train_files = os.listdir(image_path)
train_files.sort()

# Load raw label images
train_label_files = os.listdir(label_path)
train_label_files.sort()

# Divide train:val = 8:2
dataset_size = len(train_files)
train_size = int(dataset_size * 0.8)

# Create processed scar image for train
for idx in range(train_size): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])
    
    # Save normalized image
    cv2.imwrite(os.path.join(train_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(train_dir, f'label_{idx:03d}.png'), label_image)

# Create processed scar image for validation
for val_idx, idx in enumerate(range(train_size, dataset_size)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(train_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(train_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(val_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(val_dir, f'label_{idx:03d}.png'), label_image)

##### Test

In [ ]:
image_path = MODEL_PATH + RAW_TEST_SET_PATH + IMAGES_PATH
label_path = MODEL_PATH + RAW_TEST_SET_PATH + LABELS_PATH

# Load raw train images
os.chdir(image_path)
test_files = os.listdir(image_path)
test_files.sort()

# Load raw label images
os.chdir(label_path)
test_label_files = os.listdir(label_path)
test_label_files.sort()

# Create processed scar image for train
for idx in range(len(test_files)): 
    # Access train set
    os.chdir(image_path)
    image = cv2.imread(test_files[idx])

    # Access label set
    os.chdir(label_path)
    label_image = cv2.imread(test_label_files[idx])

    # Get grayscale image
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Cut black boundary on value image
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    x, y, width, height = cv2.boundingRect(contours[0])

    image = image[y:y+height, x:x+width]
    label_image = label_image[y:y+height, x:x+width]

    # Equalize histogram used by LAB and BGR color space for make enhanced image 
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_image, a_image, b_image = cv2.split(lab_image)

    l_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(4, 4))
    a_clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(2, 2))
    l_image = l_clahe.apply(l_image)
    a_image = a_clahe.apply(a_image)

    lab_image = cv2.merge((l_image, a_image, b_image))
    enhanced_image = cv2.cvtColor(lab_image, cv2.COLOR_LAB2BGR)
    
    # Clustering use K-means
    cluster_data = enhanced_image.reshape((-1, 3)).astype(np.float32)
    k = 3
    criteria = (cv2.TERM_CRITERIA_EPS|cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0) # 최대 10번 반복하고 1픽셀 이하로 움직이면 종료
    ret, c_label, center = cv2.kmeans(cluster_data, 
                                    k, 
                                    None, 
                                    criteria,
                                    10,
                                    cv2.KMEANS_RANDOM_CENTERS)
    
    center = np.uint8(center)
    kmeans_image = center[c_label.flatten()]
    kmeans_image = kmeans_image.reshape((enhanced_image.shape))
    kmeans_image = cv2.cvtColor(kmeans_image, cv2.COLOR_BGR2GRAY)

    # Pad image
    pad_width = int((NORM_INPUT_W_SIZE - width) / 2)
    pad_height = int((NORM_INPUT_H_SIZE - height) / 2)

    pad_top = pad_height
    pad_bottom = pad_height
    pad_left = pad_width
    pad_right = pad_width

    total_height = height + (pad_height * 2)
    total_width = width + (pad_width * 2)
        
    if total_height > NORM_INPUT_H_SIZE:
        pad_top = pad_top - (total_height - NORM_INPUT_W_SIZE)
    elif total_height < NORM_INPUT_H_SIZE:
        pad_bottom = pad_bottom + (NORM_INPUT_W_SIZE - total_height)

    if total_width > NORM_INPUT_W_SIZE:
        pad_left = pad_left - (total_width - NORM_INPUT_H_SIZE)
    elif total_width < NORM_INPUT_W_SIZE:
        pad_right = pad_right + (NORM_INPUT_H_SIZE - total_width)

    kmeans_image = cv2.copyMakeBorder(kmeans_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,     # method
                               value=[0, 0, 0])         # constant value
    label_image = cv2.copyMakeBorder(label_image,
                               pad_top,
                               pad_bottom,
                               pad_left,
                               pad_right,
                               cv2.BORDER_CONSTANT,
                               value=[0, 0, 0])

    # Save normalized image
    cv2.imwrite(os.path.join(test_dir, f'scar_{idx:03d}.png'), kmeans_image)
    cv2.imwrite(os.path.join(test_dir, f'label_{idx:03d}.png'), label_image)